In [20]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.2f}'.format

In [3]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [4]:
def search_pay_func(mes: int) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    

    param_dia_ini = datetime(2024, mes, 1)
    param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)

    search_pay_acc = [
        "&", "&",
            ("partner_type", "=", "customer"),
            ("is_internal_transfer", "=", False),
        "&",
            ("date", ">=", param_dia_ini.strftime('%Y-%m-%d')),
            ("date", "<=", param_dia_fin.strftime('%Y-%m-%d')),
        ]


    param_dia_ini_pos = param_dia_ini + timedelta(hours=7)
    param_dia_fin_pos = param_dia_fin + timedelta(hours=31) - timedelta(seconds= 1)
    
    search_pay_pos = [
        "&",
            ("payment_date", ">=", param_dia_ini_pos.strftime('%Y-%m-%d %H:%M:%S')),
            ("payment_date", "<=", param_dia_fin_pos.strftime('%Y-%m-%d %H:%M:%S')),
        ]
    

    return search_pay_acc, search_pay_pos

In [5]:
def api_call_pay_acc_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    search_pay_acc = search_pay[0]


    pay_acc_fields = [
                    'name',
                    'date',
                    'amount_total',
                    'reconciled_invoice_ids',
                    'pos_session_id'
                    ]

    pay_acc_ids = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'search', [search_pay_acc])
    pay_acc_json = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'read', [pay_acc_ids], {'fields': pay_acc_fields})

    
    return pay_acc_json

In [6]:
def pay_acc_df_func(pay_acc_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_acc = []
    data_fact_ids = []

    for pay in pay_acc_json:
        if not pay['pos_session_id']:
            new = {}
            new['id'] = pay['id']
            new['name'] = pay['name']
            new['date'] = pay['date']
            new['amount_total'] = pay['amount_total']
            new['pay_fact_docs'] = pay['reconciled_invoice_ids'] if pay['reconciled_invoice_ids'] else pd.NA
            
            data_fact_ids += pay['reconciled_invoice_ids']

            data_pay_acc.append(new)

    pay_acc_df = pd.DataFrame(data_pay_acc)


    pay_acc_df['date'] = pd.to_datetime(pay_acc_df['date'], format='%Y-%m-%d')
    
    return pay_acc_df, data_fact_ids

In [7]:
def api_call_pay_pos_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    search_pay_pos = search_pay[1]


    pay_pos_fields = [
                    'pos_order_id',
                    'payment_date',
                    'amount',
                    'payment_method_id'
                    ]

    pay_pos_ids = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'search', [search_pay_pos])
    pay_pos_json = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'read', [pay_pos_ids], {'fields': pay_pos_fields})
    

    return pay_pos_json

In [8]:
def pay_pos_df_func(pay_pos_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_pos = []
    data_pos_ids = set()

    for pay in pay_pos_json:
        if pay['payment_method_id'][0] != 6:
            new = {}
            new['pay_pos_id'] = pay['id']
            new['pos_order_id'] = pay['pos_order_id'][0]
            new['pos_order_name'] = pay['pos_order_id'][1]
            new['date'] = pay['payment_date']
            new['amount'] = pay['amount']
        
            data_pos_ids.add(pay['pos_order_id'][0])
            data_pay_pos.append(new)

    pay_pos_df1 = pd.DataFrame(data_pay_pos)
    pay_pos_df = pay_pos_df1.groupby('pos_order_id').agg({'pay_pos_id':['first'], 'pos_order_name':['first'], 'date':['first'], 'amount':['sum']}).reset_index().iloc[:,[1,0,2,3,4]]
    pay_pos_df.columns = ['pay_pos_id', 'pos_order_id', 'pos_order_name', 'date', 'amount']

    pay_pos_df['date'] = pd.to_datetime(pay_pos_df['date'], format='%Y-%m-%d %H:%M:%S')
    
    return pay_pos_df, list(data_pos_ids)

In [9]:
def api_call_pos_order_func(api_params: dict, data_pos_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    pos_order_fields = [
                    'account_move',
                    'partner_id',
                    'session_move_id',
                    'amount_total'
                    ]

    pos_order_json = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'read', [data_pos_ids], {'fields': pos_order_fields})

    return pos_order_json

In [10]:
def pos_order_func(pos_order_json: list[dict], data_fact_ids: list[int]) -> list[list[int]]:
    
    data_session_move_ids = [] 
    data_partner_ids = [] 

    for pos in pos_order_json:
        
        if pos['account_move']:
            data_fact_ids.append(pos['account_move'][0])
        
        data_partner_ids.append(pos['partner_id'][0])
        data_session_move_ids.append(pos['session_move_id'][0])
    
    
    return data_fact_ids, data_session_move_ids, data_partner_ids

In [11]:
def api_call_account_line_func(api_params: dict, data_session_move_ids: list[int], data_partner_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    account_line_sessions_fields = [
                    'matching_number'
                    ]

    account_line_sessions_ids = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'search', [["&", ("move_id", "in", data_session_move_ids), ("partner_id", "in", data_partner_ids)]])
    account_line_sessions_json = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'read', [account_line_sessions_ids], {'fields': account_line_sessions_fields})


    data_matching_number = [] 

    for pos in account_line_sessions_json:
        if pos['matching_number']:
            data_matching_number.append(pos['matching_number'])

    
    account_line_matching_number_fields = [
                    'move_id'
                    ]

    account_line_matching_number_ids = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'search', [['&', ("move_type", "=", "out_invoice"), ("matching_number", "in", data_matching_number)]])
    account_line_matching_number_json = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'read', [account_line_matching_number_ids], {'fields': account_line_matching_number_fields})


    return account_line_matching_number_json

In [12]:
def data_fact_ids_func(account_line_json: list[dict], data_fact_ids: list[int]) -> list[list[int]]:
    
    for line in account_line_json:
        data_fact_ids.append(line['move_id'][0])

    return list(set(data_fact_ids))

In [13]:
def api_call_acccount_docs_func(api_params: dict, data_fact_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    data_fact_fields = [
                    'name',
                    'partner_id',
                    'date',
                    'invoice_payments_widget',
                    'amount_total',
                    'amount_residual'
                    ]

    fact_doc_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [data_fact_ids], {'fields': data_fact_fields})

    return fact_doc_json

In [14]:
# def pay_pos_df_func(pay_pos_json: list[dict]) -> list[pd.DataFrame, int]:
    
#     data_pay_pos = []
#     data_pos_ids = set()

#     for pay in pay_pos_json:
#         if pay['payment_method_id'][0] != 6:
#             new = {}
#             new['pay_pos_id'] = pay['id']
#             new['pos_order_id'] = pay['pos_order_id'][0]
#             new['pos_order_name'] = pay['pos_order_id'][1]
#             new['date'] = pay['payment_date']
#             new['amount'] = pay['amount']
        
#             data_pos_ids.add(pay['pos_order_id'][0])
#             data_pay_pos.append(new)

#     pay_pos_df1 = pd.DataFrame(data_pay_pos)
#     pay_pos_df = pay_pos_df1.groupby('pos_order_id').agg({'pay_pos_id':['first'], 'pos_order_name':['first'], 'date':['first'], 'amount':['sum']}).reset_index().iloc[:,[1,0,2,3,4]]
#     pay_pos_df.columns = ['pay_pos_id', 'pos_order_id', 'pos_order_name', 'date', 'amount']

#     pay_pos_df['date'] = pd.to_datetime(pay_pos_df['date'], format='%Y-%m-%d %H:%M:%S')
    
#     return pay_pos_df, list(data_pos_ids)

In [15]:
api_params = api_params_func(True)
search_pay = search_pay_func(4)

pay_acc_json = api_call_pay_acc_func(api_params, search_pay)
pay_acc_df, data_fact_ids3 = pay_acc_df_func(pay_acc_json)

pay_pos_json = api_call_pay_pos_func(api_params, search_pay)
pay_pos_df, data_pos_ids = pay_pos_df_func(pay_pos_json)

pos_order_json = api_call_pos_order_func(api_params, data_pos_ids)
data_fact_ids2, data_session_move_ids, data_partner_ids = pos_order_func(pos_order_json, data_fact_ids3)

account_line_json = api_call_account_line_func(api_params, data_session_move_ids, data_partner_ids)
data_fact_ids = data_fact_ids_func(account_line_json, data_fact_ids2)

fact_doc_json = api_call_acccount_docs_func(api_params, data_fact_ids)

In [17]:
data_pay_fact = []

for fact in fact_doc_json:
    if fact['invoice_payments_widget']:
        for pay in fact['invoice_payments_widget']['content']:
             if 'Facturas' not in pay['journal_name'] and datetime.strptime(pay['date'], '%Y-%m-%d').month == 4:
                new = {}
                new['fac_doc_id'] = fact['id']
                new['fac_doc_name'] = fact['name']
                new['fac_doc_cliente'] = fact['partner_id'][1]
                new['fac_doc_date'] = fact['date']
                new['fac_doc_total'] = fact['amount_total']
                new['fac_doc_deuda'] = fact['amount_residual']
                new['pay_journal'] = pay['journal_name']
                new['pay_amount'] = pay['amount']
                new['pay_date'] = pay['date']
                new['pay_pos'] = pay['pos_payment_name'] if pay['pos_payment_name'] else pd.NA
    
                data_pay_fact.append(new)

pay_fact_df = pd.DataFrame(data_pay_fact)
pay_fact_df['fac_doc_date'] = pd.to_datetime(pay_fact_df['fac_doc_date'], format='%Y-%m-%d')
pay_fact_df['pay_date'] = pd.to_datetime(pay_fact_df['pay_date'], format='%Y-%m-%d')

In [26]:
pay_fact_df[['pay_pos', 'pay_amount']].groupby('pay_pos').agg({'pay_amount': ['count', 'sum']})

pay_amount           
                                         count        sum
pay_pos                                                  
Efectivo A1                               1032 375,707.14
Efectivo A2                               1194 424,217.70
Tarjeta Crédito A1                         136 149,800.80
Tarjeta Crédito A2                         220 305,872.22
Tarjeta Débito A1                          458 435,937.31
Tarjeta Débito A2                          522 500,362.30
Transferencia Banco Sant92000702524        144 792,272.16
Transferencia Banco Scot23900002860          6   6,510.51

In [29]:
# pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Efectivo A1'].to_excel('Efectivo_A1.xlsx')
pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Efectivo A1']

In [27]:
pay_fact_df[['pay_journal', 'pay_amount']].groupby('pay_journal').agg({'pay_amount': ['count', 'sum']})

pay_amount             
                           count          sum
pay_journal                                  
Banco Bana70103464895         11   116,869.17
Banco Sant65507231316        510 3,190,960.26
Banco Sant92000702524         77   294,498.47
Punto De Venta A1           1702 1,170,460.73
Punto De Venta A2           2040 1,866,941.18

In [18]:
pay_fact_df

,fac_doc_id,fac_doc_name,fac_doc_cliente,fac_doc_date,fac_doc_total,fac_doc_deuda,pay_journal,pay_amount,pay_date,pay_pos
0,65537,F2-CC/2024/07628,CUSTOM DESIGN AND CONSTRUCTION,2024-04-19,334.64,0.0,Punto De Venta A2,334.64,2024-04-19,Efectivo A2
1,57348,F1-CC/2024/05122,Emilio Costich Perez,2024-04-08,73.66,0.0,Punto De Venta A1,73.66,2024-04-08,Efectivo A1
2,65541,F1-CC/2024/05828,LGF ARQUITECTURA,2024-04-19,78.42,0.0,Punto De Venta A1,69.50,2024-04-19,Tarjeta Débito A1
3,65541,F1-CC/2024/05828,LGF ARQUITECTURA,2024-04-19,78.42,0.0,Punto De Venta A1,8.92,2024-04-19,Efectivo A1
4,57352,F1-CC/2024/05123,MICHEL ANGEL AGUIRRE PACHECO,2024-04-08,1389.91,0.0,Punto De Venta A1,1389.91,2024-04-08,Tarjeta Débito A1
...,...,...,...,...,...,...,...,...,...,...
4335,65523,F2-CC/2024/07626,Mostrador,2024-04-19,475.14,0.0,Punto De Venta A2,475.14,2024-04-19,Efectivo A2
4336,57334,F1-CC/2024/05121,MY WAY TOO,2024-04-08,78.76,0.0,Punto De Venta A1,78.76,2024-04-08,Efectivo A1
4337,65527,F2-CC/2024/07627,CUSTOM DESIGN AND CONSTRUCTION,2024-04-19,238.70,0.0,Punto De Venta A2,238.70,2024-04-19,Efectivo A2
4338,65530,F1-CC/2024/05826,Emilio Costich Perez,2024-04-19,65.26,0.0,Punto De Venta A1,65.26,2024-04-19,Efectivo A1


In [19]:
pay_fact_df.agg({'pay_amount':['sum']})

,pay_amount
sum,6639729.81


## Pruebas